In [5]:
!pip install -U wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: wandb
    Found existing installation: wandb 0.16.0
    Uninstalling wandb-0.16.0:
      Successfully uninstalled wandb-0.16.0

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [6]:
import wandb

AttributeError: module 'wandb.sdk' has no attribute 'lib'

In [4]:
wandb.login()

NameError: name 'wandb' is not defined

In [3]:
%env WANDB_PROJECT=codellama-7b-instruct-hf-peft-fine-tuning

env: WANDB_PROJECT=codellama-7b-instruct-hf-peft-fine-tuning


In [4]:
dataset_name = "iamtarun/python_code_instructions_18k_alpaca"
split = "train[:10%]"

In [5]:
from datasets import load_dataset

dataset = load_dataset( dataset_name, split=split )

In [6]:
def prompt_instruction_format( sample ):
    
  return f"""### Instruction:
    Use the Task below and the Input given to write the Response, which is programmatic instruction that can solve the following Task:

    ### Task:
    {sample['instruction']}

    ### Input:
    {sample['input']}

    ### Response:
    {sample['output']}
    """

In [9]:
# import pandas as pd
# from datasets import load_dataset, Dataset
# 
# #Load the dataset from the HuggingFace Hub
# rd_ds = load_dataset( "xiyuez/red-dot-design-award-product-description" )
# 
# #Convert to pandas dataframe for convenient processing
# rd_df = pd.DataFrame( rd_ds[ 'train' ] )
# 
# #Combine the two attributes into an instruction string
# rd_df[ 'instruction' ] = 'Create a detailed description for the following product: ' + rd_df[ 'product' ] + ', belonging to category: ' + rd_df[ 'category' ]
# 
# rd_df = rd_df[ [ 'instruction', 'description' ] ]
# 
# #Get a 5000 sample subset for fine-tuning purposes
# rd_df_sample = rd_df.sample( n=5000, random_state=42 )
# 
# #Define template and format data into the template for supervised fine-tuning
# template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
# 
# ### Instruction:
# 
# {}
# 
# ### Response:\n"""
# 
# rd_df_sample[ 'prompt' ] = rd_df_sample[ "instruction" ].apply( lambda x: template.format( x ) )
# rd_df_sample.rename( columns={ 'description': 'response' }, inplace=True )
# rd_df_sample[ 'response' ] = rd_df_sample[ 'response' ] + "\n### End"
# rd_df_sample = rd_df_sample[ [ 'prompt', 'response' ] ]
# 
# rd_df_sample[ 'text' ] = rd_df_sample[ "prompt" ] + rd_df_sample[ "response" ]
# rd_df_sample.drop( columns=[ 'prompt', 'response' ], inplace=True )

In [10]:
# import pandas as pd
# 
# dataset = Dataset.from_pandas( rd_df_sample ).train_test_split( test_size=0.05, seed=42 )
# dataset

In [15]:
import os
os.chdir( "/var/model" )

In [16]:
!ls -alh

total 31G
drwxrwxr-x 5 1001 1001 4.0K Nov 11 03:08 .
drwxr-xr-x 1 root root 4.0K Nov 14 18:07 ..
drwxr-xr-x 2 root root 4.0K Nov 11 03:01 .ipynb_checkpoints
-rw-rw-r-- 1 1001 1001 6.9K Nov  5 17:36 LICENSE
-rw-rw-r-- 1 1001 1001 6.1K Nov  5 17:36 README.md
-rw-rw-r-- 1 1001 1001 4.7K Nov  5 17:36 USE_POLICY.md
drwxr-xr-x 2 root root 4.0K Nov 10 21:49 checkpoints
-rw-r--r-- 1 root root 9.3K Nov 11 02:12 code-llama-instruct-7b-peft.ipynb
-rw-r--r-- 1 1001 1001  97K Nov 11 03:08 code-llama-instruct-7b.ipynb
-rw-rw-r-- 1 1001 1001  646 Nov  5 17:36 config.json
-rw-r--r-- 1 root root 4.1G Jun 26 07:24 cuda_12.2.0_535.54.03_linux.run
-rw-rw-r-- 1 1001 1001  116 Nov  5 17:36 generation_config.json
-rw-rw-r-- 1 1001 1001 9.3G Nov  5 17:42 model-00001-of-00002.safetensors
-rw-rw-r-- 1 1001 1001 3.3G Nov  5 17:40 model-00002-of-00002.safetensors
-rw-rw-r-- 1 1001 1001  25K Nov  5 17:36 model.safetensors.index.json
drwxr-xr-x 5 root root 4.0K Nov 14 18:34 output
-rw-rw-r-- 1 1001 1001 4.6G Nov  5

In [17]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model_path = 'codellama/CodeLlama-7b-Instruct-hf'
tokenizer = AutoTokenizer.from_pretrained( "." )
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.padding_side = 'right'

# *DOES NOT* force the model to be loaded entirely onto the second GPU
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

base_model = AutoModelForCausalLM.from_pretrained(
    ".", quantization_config=bnb_config, device_map="auto", low_cpu_mem_usage=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:

#Pass in a prompt and infer with the model
prompt = 'Q: Create a detailed description for the following product: Corelogic Smooth Mouse, belonging to category: Optical Mouse\nA:'

device = "cuda:0"
inputs = tokenizer( prompt, return_tensors="pt" ).to( device )

generation_output = model.generate(
    input_ids=inputs[ "input_ids" ],
    attention_mask=inputs[ "attention_mask" ],
    max_new_tokens=128
)

print( tokenizer.decode( generation_output[ 0 ] ) )

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>Q: Create a detailed description for the following product: Corelogic Smooth Mouse, belonging to category: Optical Mouse
A: Corelogic Smooth Mouse is a high-quality optical mouse that is designed to provide a smooth and precise experience for users. It is a wireless mouse that is designed to be used with a computer or other device. The mouse has a sleek and modern design that is easy to use and looks great on any desk. It has a 12,000-dot optical sensor that allows for precise movement and a 12-inch optical sensor that provides a smooth and precise experience. The mouse also has a 12-inch optical sensor that provides a smooth and precise experience. It has a 1


In [9]:
from peft import LoraConfig, get_peft_config, PeftModel, PeftConfig, get_peft_model

lora_config = LoraConfig(
    r=16, 
    lora_alpha=32, 
    target_modules=[ "q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj" ], 
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

In [10]:
model = get_peft_model( model, lora_config )
model.print_trainable_parameters()

trainable params: 39,976,960 || all params: 6,778,523,648 || trainable%: 0.5897590991188056


## Set up training arguments

In [11]:
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = 'adamw_hf'
learning_rate = 1e-5
max_grad_norm = 0.3
warmup_ratio = 0.03
lr_scheduler_type = "linear"

In [12]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="output",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    num_train_epochs = 3.0,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [13]:
from trl import SFTTrainer

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [16]:
trainer = SFTTrainer(
    model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    dataset_text_field="text",
    max_seq_length=256,
    args=training_args,
)
#Upcast layer norms to torch.bfloat16 for stability
for name, module in trainer.model.named_modules():
  if "norm" in name:
    module = module.to(torch.bfloat16)
    print( "Up casting to float32:", name )

Map:   0%|          | 0/4750 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Up casting to float32: base_model.model.model.layers.0.input_layernorm
Up casting to float32: base_model.model.model.layers.0.post_attention_layernorm
Up casting to float32: base_model.model.model.layers.1.input_layernorm
Up casting to float32: base_model.model.model.layers.1.post_attention_layernorm
Up casting to float32: base_model.model.model.layers.2.input_layernorm
Up casting to float32: base_model.model.model.layers.2.post_attention_layernorm
Up casting to float32: base_model.model.model.layers.3.input_layernorm
Up casting to float32: base_model.model.model.layers.3.post_attention_layernorm
Up casting to float32: base_model.model.model.layers.4.input_layernorm
Up casting to float32: base_model.model.model.layers.4.post_attention_layernorm
Up casting to float32: base_model.model.model.layers.5.input_layernorm
Up casting to float32: base_model.model.model.layers.5.post_attention_layernorm
Up casting to float32: base_model.model.model.layers.6.input_layernorm
Up casting to float32: 

In [17]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:423: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a CodeLlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=891, training_loss=1.8714779769263818, metrics={'train_runtime': 1413.0161, 'train_samples_per_second': 10.085, 'train_steps_per_second': 0.631, 'total_flos': 1.0319356998043238e+17, 'train_loss': 1.8714779769263818, 'epoch': 3.0})

In [18]:
model.save_adapter( "output", "foo_adapter" )

AttributeError: 'LlamaForCausalLM' object has no attribute 'save_adapter'

In [19]:

#Pass in a prompt and infer with the model
prompt = 'Q: Create a detailed description for the following product: Corelogic Smooth Mouse, belonging to category: Optical Mouse\nA:'

device = "cuda:0"
inputs = tokenizer( prompt, return_tensors="pt" ).to( device )

generation_output = model.generate(
    input_ids=inputs[ "input_ids" ],
    attention_mask=inputs[ "attention_mask" ],
    max_new_tokens=128
)

print( tokenizer.decode( generation_output[ 0 ] ) )

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


RuntimeError: expected mat1 and mat2 to have the same dtype, but got: float != c10::Half